# Project 4: WeRateDogs data wrangling

## Table of contents

<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
    <ul>
        <li><a href="#gathering">Data gathering</a></li>
        <li><a href="#assessment">Data assessment</a></li>
        <li><a href="#cleaning">Data cleaning</a></li>
    </ul>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<li><a href="#limitations">Limitations</a></li>
</ul>

<a id='intro'></a>
## Introduction 

In this project, we will be wrangling, analysing and visualizing  the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates), also known as [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs). After gathering this data from Twitter's api using [Tweepy](https://www.tweepy.org/), we will be assessing and cleaning it. Then, we will be finding useful insights from our cleaned data and producing some visualizations. The last step but not the least will is making a report of our results. It will be done in **act\_report.pdf** file associated to the project.

<a id='wrangling'></a>
## Data wrangling 

In [196]:
# importing libraries
import pandas as pd
import numpy as np
import tweepy 
import os
import time
import json
import requests
from PIL import Image
from io import BytesIO
from sqlalchemy import create_engine
import sqlite3

<a id='gathering'></a>
### Data gathering

#### `1.` WeRateDogs archive

In [104]:
# load WeRateDogs archive into dataframe
archive = pd.read_csv('../data/twitter-archive-enhanced.csv')
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### `2.` The tweet image predictions.

In [105]:
# download 'image_predictions.tsv' into data folder
folder_path = '../data'
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)
with open(os.path.join(folder_path, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)        

In [106]:
# check if file in data folder
os.listdir(folder_path)

['image-predictions.tsv', 'twitter-archive-enhanced.csv', 'tweet_json.txt']

In [107]:
# load 'image_predictions.tsv' into dataframe
image_predictions = pd.read_csv('../data/image-predictions.tsv', sep='\t')
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### `3.` Gather retweet and favorite counts from Tweepy.

In [108]:
# WeRateDogs tweet ids
tweet_id_list = list(archive.tweet_id)
tweet_id_list

[892420643555336193,
 892177421306343426,
 891815181378084864,
 891689557279858688,
 891327558926688256,
 891087950875897856,
 890971913173991426,
 890729181411237888,
 890609185150312448,
 890240255349198849,
 890006608113172480,
 889880896479866881,
 889665388333682689,
 889638837579907072,
 889531135344209921,
 889278841981685760,
 888917238123831296,
 888804989199671297,
 888554962724278272,
 888202515573088257,
 888078434458587136,
 887705289381826560,
 887517139158093824,
 887473957103951883,
 887343217045368832,
 887101392804085760,
 886983233522544640,
 886736880519319552,
 886680336477933568,
 886366144734445568,
 886267009285017600,
 886258384151887873,
 886054160059072513,
 885984800019947520,
 885528943205470208,
 885518971528720385,
 885311592912609280,
 885167619883638784,
 884925521741709313,
 884876753390489601,
 884562892145688576,
 884441805382717440,
 884247878851493888,
 884162670584377345,
 883838122936631299,
 883482846933004288,
 883360690899218434,
 883117836046

In [198]:
# retrieve api credential from we-rate-dogs sqlite database located in ../data/we-rate-dogs.db
db = sqlite3.connect('../data/we-rate-dogs.db')
query = 'SELECT * FROM auth;'
cursor = db.execute(f'{query}')
for consumer_key, consumer_secret, access_token, access_secret in cursor.fetchall():
    consumer_key = consumer_key
    consumer_secret = consumer_secret
    access_token = access_token,
    access_secret = access_secret

In [200]:
# instantiate a tweepy object 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# File path to save tweets' content
tweet_json_path = '../data/tweet_json.txt'
# Quering twitter ids in 'twitter_ids_list'
for tweet_id in tweet_id_list: 
    try:
        start_time = time.time() # start timing request 
        tweet = api.get_status(tweet_id, tweet_mode="extended")
        end_time = time.time() # end timing request
        print(f"Tweet id: {tweet_id}. Request duration: {end_time - start_time}") 
        # Writing Twitter Json file into ../data/tweet_json.txt file.
        tweet_json =[]
        tweet_json.append({tweet.id: f'{tweet.retweet_count} {tweet.favorite_count}'}) 
        with open(tweet_json_path, mode='a', encoding='utf-8') as file:
            json.dump(tweet_json[0], file)
            file.write("\n")
    except:
        print(f"⚠️ Warning ⚠️ Tweet associated with the id {tweet_id} has been deleted")

Tweet id: 892420643555336193. Request duration: 7.184101343154907
Tweet id: 892177421306343426. Request duration: 0.8641998767852783
Tweet id: 891815181378084864. Request duration: 0.875007152557373


In [110]:
# Strip a set of characters from a string, Source: https://www.w3resource.com/python-exercises/string/python-data-type-string-exercise-41.php
def strip_chars(str, chars):
    return "".join(c for c in str if c not in chars)

In [111]:
# Read data from tweet_json.txt and append it to tweet_json_list using this template: f'{tweet_id} {retweet_count} {favorite_count}'
tweet_json_list = []

with open('../data/tweet_json.txt', mode='r+', encoding='utf-8') as file:
    for line in file.readlines():
        line = strip_chars(line, '{}":\n')
        tweet_json_list.append(line)

In [112]:
# create tweet_json  dataframe and populate its column 'tweet' with tweet_json_list content.
tweet_json = pd.DataFrame(tweet_json_list, columns=['tweet'])

In [113]:
# Create tweet_id, retweet_count and favorite_count columns with the right fit data
tweet_json['tweet_id'] = tweet_json.tweet.str.extract(r'([0-9]+(?=\s))').astype(int)

tweet_json['retweet_count'] = tweet_json.tweet.str.extract(r'((?<=\s)[0-9]+(?=\s))').astype(int)

tweet_json['favorite_count'] = tweet_json.tweet.str.extract(r'[\s]{1}([0-9]+)$').replace(' ','').astype(int)

In [114]:
# drop column tweet_json
tweet_json.drop('tweet', axis='columns', inplace=True)

In [115]:
tweet_json.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7487,35458
1,892177421306343426,5557,30696
2,891815181378084864,3680,23088
3,891689557279858688,7664,38741
4,891327558926688256,8265,37024


<a id='assessment'></a>
### Data assessment

#### `archive` table

In [116]:
archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


In [117]:
# df_twitter's columns
list(archive)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [118]:
archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [141]:
list(archive.name)

['Phineas',
 'Tilly',
 'Archie',
 'Darla',
 'Franklin',
 'None',
 'Jax',
 'None',
 'Zoey',
 'Cassie',
 'Koda',
 'Bruno',
 'None',
 'Ted',
 'Stuart',
 'Oliver',
 'Jim',
 'Zeke',
 'Ralphus',
 'Canela',
 'Gerald',
 'Jeffrey',
 'such',
 'Canela',
 'None',
 'None',
 'Maya',
 'Mingus',
 'Derek',
 'Roscoe',
 'None',
 'Waffles',
 'None',
 'Jimbo',
 'Maisey',
 'None',
 'Lilly',
 'None',
 'Earl',
 'Lola',
 'Kevin',
 'None',
 'None',
 'Yogi',
 'Noah',
 'Bella',
 'Grizzwald',
 'None',
 'Rusty',
 'Gus',
 'Stanley',
 'Alfy',
 'Koko',
 'Rey',
 'Gary',
 'None',
 'a',
 'Elliot',
 'Louis',
 'None',
 'Bella',
 'Jesse',
 'None',
 'Romeo',
 'None',
 'Bailey',
 'Duddles',
 'Jack',
 'Emmy',
 'Steven',
 'Beau',
 'Snoopy',
 'None',
 'Shadow',
 'Terrance',
 'Shadow',
 'Emmy',
 'Aja',
 'None',
 'Penny',
 'Dante',
 'Nelly',
 'Ginger',
 'None',
 'Benedict',
 'Venti',
 'Goose',
 'Nugget',
 'None',
 'None',
 'Cash',
 'Coco',
 'Jed',
 'None',
 'Sebastian',
 'Walter',
 'None',
 'Sierra',
 'Sierra',
 'None',
 'None',
 

In [143]:
archive.name.value_counts()

None         745
a             55
Charlie       12
Cooper        11
Oliver        11
            ... 
Scruffers      1
Sprout         1
Beebop         1
Danny          1
Aiden          1
Name: name, Length: 957, dtype: int64

In [192]:
list(archive.name.sort_values())

['Abby',
 'Abby',
 'Ace',
 'Acro',
 'Adele',
 'Aiden',
 'Aja',
 'Akumi',
 'Al',
 'Albert',
 'Albert',
 'Albus',
 'Albus',
 'Aldrick',
 'Alejandro',
 'Alexander',
 'Alexanderson',
 'Alf',
 'Alfie',
 'Alfie',
 'Alfie',
 'Alfie',
 'Alfie',
 'Alfy',
 'Alice',
 'Alice',
 'Amber',
 'Ambrose',
 'Amy',
 'Amélie',
 'Anakin',
 'Anakin',
 'Andru',
 'Andy',
 'Angel',
 'Anna',
 'Anthony',
 'Antony',
 'Apollo',
 'Aqua',
 'Archie',
 'Archie',
 'Archie',
 'Archie',
 'Arlen',
 'Arlo',
 'Arnie',
 'Arnie',
 'Arnie',
 'Arnold',
 'Arya',
 'Ash',
 'Ash',
 'Asher',
 'Ashleigh',
 'Aspen',
 'Aspen',
 'Astrid',
 'Astrid',
 'Atlas',
 'Atlas',
 'Atticus',
 'Atticus',
 'Aubie',
 'Augie',
 'Autumn',
 'Ava',
 'Ava',
 'Axel',
 'Axel',
 'Bailey',
 'Bailey',
 'Bailey',
 'Bailey',
 'Bailey',
 'Bailey',
 'Bailey',
 'Baloo',
 'Baloo',
 'Balto',
 'Balto',
 'Banditt',
 'Banjo',
 'Barclay',
 'Barney',
 'Baron',
 'Barry',
 'Batdog',
 'Bauer',
 'Baxter',
 'Baxter',
 'Bayley',
 'BeBe',
 'Bear',
 'Bear',
 'Bear',
 'Bear',
 'Beau

In [140]:
archive[['name', 'text']]

,name,text
0,Phineas,This is Phineas. He's a mystical boy. Only eve...
1,Tilly,This is Tilly. She's just checking pup on you....
2,Archie,This is Archie. He is a rare Norwegian Pouncin...
3,Darla,This is Darla. She commenced a snooze mid meal...
4,Franklin,This is Franklin. He would like you to stop ca...
...,...,...
2351,None,Here we have a 1949 1st generation vulpix. Enj...
2352,a,This is a purebred Piers Morgan. Loves to Netf...
2353,a,Here is a very happy pup. Big fan of well-main...
2354,a,This is a western brown Mitsubishi terrier. Up...


In [138]:
archive[['name', 'text']].sort_values('name')

,name,text
1035,Abby,This is Abby. She got her face stuck in a glas...
1021,Abby,Meet Abby. She's incredibly distracting. Just ...
938,Ace,This is Ace. He's a window washer. One of the ...
1933,Acro,This is Acro. You briefly see her out of the c...
1327,Adele,This is Adele. Her tongue flies out of her mou...
...,...,...
1031,very,We only rate dogs. Pls stop sending in non-can...
773,very,RT @dog_rates: We only rate dogs. Pls stop sen...
1097,very,We only rate dogs. Please stop sending in non-...
819,very,We only rate dogs. Pls stop sending in non-can...


In [17]:
# Match names that start with a lower case letter
archive[archive.name.str.match('[^A-Z]')].name.unique()

array(['such', 'a', 'quite', 'not', 'one', 'incredibly', 'mad', 'an',
       'very', 'just', 'my', 'his', 'actually', 'getting', 'this',
       'unacceptable', 'all', 'old', 'infuriating', 'the', 'by',
       'officially', 'life', 'light', 'space'], dtype=object)

In [132]:
# wrong name 'my' 
archive.query('name == "my"')['text']

852    This is my dog. Her name is Zoey. She knows I'...
Name: text, dtype: object

In [133]:
# wrong name Al
archive.query('name == "Al"')['text']

820    Meet Al Cabone. He's a gangsta puppa. Rather h...
Name: text, dtype: object

In [134]:
# wrong name O
archive.query('name == "O"')['text']

775    This is O'Malley. That is how he sleeps. Doesn...
Name: text, dtype: object

In [18]:
# Match single letter names
archive[archive.name.str.contains(r'^[A-Z]{1}$')].name.unique()

array(['O'], dtype=object)

In [136]:
list(archive.source)

['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 '<a href="http://twitter.com/download/ipho

In [135]:
archive[['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
0,This is Phineas. He's a mystical boy. Only eve...,13,10
1,This is Tilly. She's just checking pup on you....,13,10
2,This is Archie. He is a rare Norwegian Pouncin...,12,10
3,This is Darla. She commenced a snooze mid meal...,13,10
4,This is Franklin. He would like you to stop ca...,12,10
...,...,...,...
2351,Here we have a 1949 1st generation vulpix. Enj...,5,10
2352,This is a purebred Piers Morgan. Loves to Netf...,6,10
2353,Here is a very happy pup. Big fan of well-main...,9,10
2354,This is a western brown Mitsubishi terrier. Up...,7,10


#### `image_predictions` table

In [19]:
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [20]:
# df_image_predictions' columns
list(image_predictions)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

In [21]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


#### `tweet_json` table

In [22]:
tweet_json

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7487,35458
1,892177421306343426,5557,30696
2,891815181378084864,3680,23088
3,891689557279858688,7664,38741
4,891327558926688256,8265,37024
...,...,...,...
2326,666049248165822465,40,96
2327,666044226329800704,125,264
2328,666033412701032449,39,109
2329,666029285002620928,41,119


In [23]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
tweet_id          2331 non-null int64
retweet_count     2331 non-null int64
favorite_count    2331 non-null int64
dtypes: int64(3)
memory usage: 54.8 KB


### Tidiness

1. Tweets to keep in the **`archive`** table are those in **`image_predictions`** table.


2. Retweet count and favorite count in **`tweet_json`** table should in **`archive`** table.


3. Column `text` in **`archive`** table contains URLs that should be removed since there is a column **expanded_urls** that can direct to the tweet. 


4. Dog Stages `doggo, floofer, pupper, puppo`  should be stored in `stage` column in **`archive`** table.

### Quality

##### `archive` table 

1. `timestamp` data type should be datatime.


2. `expanded_urls`, missing values


3. `name` **Zoey** instead of **my** 


4. `name` **Al Cabone** instead of **Al**


5. `name` **O'Malley** instead of **O**. Check single letter names.

 
6. `name` **Quizno** instead of **his**. Seems not to be a dog.


7. **a, not, one, an, just, very, actually, such, the, this, unacceptable,....** instead of **None** or names. These names seems to be lower case. 


8. `source` column as Text-formatted string instead HTML-formatted string for clarity.


9. Some `rating numerators` and `denominators` don't match with rating in text column.


10. `in_reply_to_user_id`, `in_reply_to_status_id`, `retweeted_status_id`, `retweeted_status_user_id` in **string** instead **float**. Missing values.


11. `retweeted_status_timestamp` data type should be a datatime.

<a id='cleaning'></a>
### Data cleaning

In [144]:
# make a copy of original data
archive_clean = archive.copy()
image_predictions_clean = image_predictions.copy()
tweet_json_clean = tweet_json.copy()

### Tidiness

#### `1.` 

Tweets to keep in the `archive` table are those in `image_predictions` table.

###### Define

Select rows in `archive` table based on `image_predictions` tweet_id column. 

###### Code

In [145]:
archive_clean = archive_clean[archive_clean.tweet_id.isin(image_predictions_clean.tweet_id)]

###### Test

In [146]:
archive_clean.shape[0] == image_predictions_clean.shape[0]

True

#### `2.` 

Retweet count and favorite count in `tweet_json` table should in `archive` table.

###### Define

Merge retweet_count and favorite_count columns to `archive` table joining on tweet_id

###### Code

In [147]:
archive_clean = pd.merge(archive_clean, tweet_json_clean, on='tweet_id', how='inner')

###### Test

In [148]:
all_columns = pd.Series(list(archive_clean) + list(tweet_json))
all_columns[all_columns.duplicated()]

19          tweet_id
20     retweet_count
21    favorite_count
dtype: object

#### `3.` 

Column text in `archive` table contains URLs that should be removed since there is a column expanded_urls that can direct to the tweet. 

###### Define

Match url pattern in the text and delete it. 

###### Code

In [149]:
archive_clean.text = archive_clean.text.str.extract(r'([\w\s\W]+(?=\shttp[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+))')

###### Test

In [194]:
archive_clean.text

0       This is Phineas. He's a mystical boy. Only eve...
1       This is Tilly. She's just checking pup on you....
2       This is Archie. He is a rare Norwegian Pouncin...
3       This is Darla. She commenced a snooze mid meal...
4       This is Franklin. He would like you to stop ca...
                              ...                        
2054    Here we have a 1949 1st generation vulpix. Enj...
2055    This is a purebred Piers Morgan. Loves to Netf...
2056    Here is a very happy pup. Big fan of well-main...
2057    This is a western brown Mitsubishi terrier. Up...
2058    Here we have a Japanese Irish Setter. Lost eye...
Name: text, Length: 2058, dtype: object

In [151]:
archive_clean.text[0]

"This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10"

#### `4.` 

Dog Stages (doggo, floofer, pupper, puppo)  should be stored in **stage** column in `archive` table.

#### Define

Melt doggo, floofer, pupper, puppo to a column **stage**

#### Code

In [152]:
archive_clean['stage'] = (archive_clean['doggo'] + archive_clean['floofer'] + archive_clean['pupper'] + archive_clean['puppo']).str.strip()

In [153]:
archive_clean.stage = archive_clean.stage.apply(lambda x: x.replace('None','').strip())

In [154]:
archive_clean.stage.value_counts()

                1741
pupper           210
doggo             65
puppo             23
doggopupper       11
floofer            7
doggofloofer       1
doggopuppo         1
Name: stage, dtype: int64

In [155]:
archive_clean.stage = archive_clean.stage.replace({'':'None', 'doggopupper': 'puppo', 'doggofloofer': 'floofer', 'doggopuppo': 'doggo'})

In [156]:
archive_clean.drop(columns=['doggo', 'floofer', 'pupper','puppo'], axis=1, inplace=True)

#### Test

In [157]:
archive_clean.shape

(2059, 16)

In [158]:
archive_clean.stage.value_counts()

None       1741
pupper      210
doggo        66
puppo        34
floofer       8
Name: stage, dtype: int64

### Quality

##### `archive` table 

#### `1.` 

`timestamp` data type should be datatime.

####  Define

Convert timestamp to datetime

#### Code

In [159]:
archive_clean.timestamp = pd.to_datetime(archive_clean.timestamp)

#### Test

In [160]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059 entries, 0 to 2058
Data columns (total 16 columns):
tweet_id                      2059 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2059 non-null datetime64[ns, UTC]
source                        2059 non-null object
text                          2059 non-null object
retweeted_status_id           72 non-null float64
retweeted_status_user_id      72 non-null float64
retweeted_status_timestamp    72 non-null object
expanded_urls                 2059 non-null object
rating_numerator              2059 non-null int64
rating_denominator            2059 non-null int64
name                          2059 non-null object
retweet_count                 2059 non-null int64
favorite_count                2059 non-null int64
stage                         2059 non-null object
dtypes: datetime64[ns, UTC](1), float64(4), int64(5), object(6)
memory usag

`2.`

`expanded_urls`, missing 59 values

#### Define

Select rows with missing values and fill them with **None**.

#### Code

In [161]:
archive_clean[archive_clean.expanded_urls.isnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage


### [Resolved] 

There is any missing value in `expanded_urls` column anymore. Done thanks to the **selection of rows in `archive` table based on** `image_predictions` **tweet_id column.**

#### Test

In [162]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059 entries, 0 to 2058
Data columns (total 16 columns):
tweet_id                      2059 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2059 non-null datetime64[ns, UTC]
source                        2059 non-null object
text                          2059 non-null object
retweeted_status_id           72 non-null float64
retweeted_status_user_id      72 non-null float64
retweeted_status_timestamp    72 non-null object
expanded_urls                 2059 non-null object
rating_numerator              2059 non-null int64
rating_denominator            2059 non-null int64
name                          2059 non-null object
retweet_count                 2059 non-null int64
favorite_count                2059 non-null int64
stage                         2059 non-null object
dtypes: datetime64[ns, UTC](1), float64(4), int64(5), object(6)
memory usag

#### `3.`

`name` **Zoey** instead of **my**

#### Define

Replace **my** by **Zoey**

#### Code

In [163]:
archive_clean.name.replace('my', 'Zoey', inplace=True)

#### Test

In [164]:
archive_clean.query('name == "my"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage


#### `4.`

`name` **Al Cabone** instead of **Al**

#### Define

Replace **Al Cabone** by **Al**

#### Code

In [165]:
archive_clean.name.replace('Al', 'Al Cabone', inplace=True)

#### Test 

In [166]:
archive_clean.query('name == "Al"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage


#### `5.` 

`name` **O'Malley** instead of **O**. Check single letter names.

#### Define 

Find single letter names and replace with replace with right names

#### Code

In [167]:
# Match single letter names
archive[archive.name.str.contains(r'^[A-Z]{1}$')].name.unique()

array(['O'], dtype=object)

In [168]:
archive_clean.name.replace('O', "O'Malley", inplace=True)

#### Test

In [169]:
archive_clean.query('name == "O"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage


#### `6.` 

`name` **Quizno** instead of **his**. Seems not to be a dog.

#### Define

Delete row with `name` **his**.

#### Code

In [170]:
# find row's index
archive_clean.query('name == "his"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage
801,748692773788876800,NaN,NaN,2016-07-01 01:40:41+00:00,"<a href=""http://twitter.com/download/iphone"" r...",That is Quizno. This is his beach. He does not...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/748692773...,10,10,his,1303,4171,doggo


In [171]:
archive_clean.drop(index=801, axis=0, inplace=True)

#### Test

In [172]:
archive_clean.query('name == "his"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage


#### `7.`

**a, not, one, an, just, very, actually, such, the, this, unacceptable,....** instead of **None** or names. These names seems to be lower case. 

#### Define

Find list of wrong names. Replace `name` by **None**

#### Code

In [173]:
# Match names that start with a lower case letter
wrong_names = archive_clean[archive_clean.name.str.match('[^A-Z]')].name.unique()
wrong_names

array(['such', 'a', 'quite', 'one', 'incredibly', 'an', 'very', 'just',
       'not', 'getting', 'this', 'unacceptable', 'all', 'infuriating',
       'the', 'actually', 'by', 'officially', 'light', 'space'],
      dtype=object)

In [174]:
archive_clean.replace(wrong_names, 'None', inplace=True)

#### Test

In [175]:
archive_clean[archive_clean.name.isin(wrong_names)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,retweet_count,favorite_count,stage


#### `8.`

`source` column as Text-formatted string instead HTML-formatted string for clarity.

#### Define

Extract text from **a** tag

#### Code

In [176]:
archive_clean.source = archive_clean.source.str.extract(r'((?<=[>])[\w\s]+(?=</a>))')

#### Test

In [177]:
archive_clean.source.head()

0    Twitter for iPhone
1    Twitter for iPhone
2    Twitter for iPhone
3    Twitter for iPhone
4    Twitter for iPhone
Name: source, dtype: object

#### `9.`

Some `rating numerators` and `denominators` don't match with rating in text column.

#### Define

Extract rating from `text` column and fill `rating_numerator` and `rating_denominator` with their right values.

#### Code 

In [178]:
archive_clean['rating'] = archive_clean.text.str.extract(r'(\d+/\d+)')
archive_clean.rating_numerator, archive_clean.rating_denominator = archive_clean.rating.str.split('/', 1).str
archive_clean.rating_numerator = archive_clean.rating_numerator.astype(int)
archive_clean.rating_denominator = archive_clean.rating_denominator.astype(int)
archive_clean.drop(columns='rating', inplace=True)

#### Test

In [179]:
list(archive_clean)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'retweet_count',
 'favorite_count',
 'stage']

In [180]:
archive_clean.rating_numerator.head()

0    13
1    13
2    12
3    13
4    12
Name: rating_numerator, dtype: int64

In [181]:
archive_clean.rating_denominator.head()

0    10
1    10
2    10
3    10
4    10
Name: rating_denominator, dtype: int64

In [182]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2058 entries, 0 to 2058
Data columns (total 16 columns):
tweet_id                      2058 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2058 non-null datetime64[ns, UTC]
source                        2058 non-null object
text                          2058 non-null object
retweeted_status_id           72 non-null float64
retweeted_status_user_id      72 non-null float64
retweeted_status_timestamp    72 non-null object
expanded_urls                 2058 non-null object
rating_numerator              2058 non-null int64
rating_denominator            2058 non-null int64
name                          2058 non-null object
retweet_count                 2058 non-null int64
favorite_count                2058 non-null int64
stage                         2058 non-null object
dtypes: datetime64[ns, UTC](1), float64(4), int64(5), object(6)
memory usag

#### `10.`

`in_reply_to_user_id`, `in_reply_to_status_id`, `retweeted_status_id`, `retweeted_status_user_id` in **string** instead **float**. Missing values.

#### Define

Convert variables into string and replace null values by **None**

#### Code

In [183]:
archive_clean.in_reply_to_status_id = archive_clean.in_reply_to_status_id.astype(str)
archive_clean.in_reply_to_status_id.replace('', 'None', inplace=True)

In [184]:
archive_clean.in_reply_to_user_id = archive_clean.in_reply_to_user_id.astype(str)
archive_clean.in_reply_to_user_id.replace('', 'None', inplace=True)

In [185]:
archive_clean.retweeted_status_id = archive_clean.retweeted_status_id.astype(str)
archive_clean.retweeted_status_id.replace('', 'None', inplace=True)

In [186]:
archive_clean.retweeted_status_user_id = archive_clean.retweeted_status_user_id.astype(str)
archive_clean.retweeted_status_user_id.replace('', 'None', inplace=True)

In [187]:
archive_clean.retweeted_status_id = archive_clean.retweeted_status_id.astype(str)
archive_clean.retweeted_status_id.replace('', 'None', inplace=True)

In [188]:
archive_clean.retweeted_status_user_id = archive_clean.retweeted_status_user_id.astype(str)
archive_clean.retweeted_status_user_id.replace('', 'None', inplace=True)

#### Test

In [189]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2058 entries, 0 to 2058
Data columns (total 16 columns):
tweet_id                      2058 non-null int64
in_reply_to_status_id         2058 non-null object
in_reply_to_user_id           2058 non-null object
timestamp                     2058 non-null datetime64[ns, UTC]
source                        2058 non-null object
text                          2058 non-null object
retweeted_status_id           2058 non-null object
retweeted_status_user_id      2058 non-null object
retweeted_status_timestamp    72 non-null object
expanded_urls                 2058 non-null object
rating_numerator              2058 non-null int64
rating_denominator            2058 non-null int64
name                          2058 non-null object
retweet_count                 2058 non-null int64
favorite_count                2058 non-null int64
stage                         2058 non-null object
dtypes: datetime64[ns, UTC](1), int64(5), object(10)
memory usage: 273.

#### `11.` 

`retweeted_status_timestamp` data type should be a datatime.

#### Define

Convert `retweeted_status_timestamp` to datetime.

#### Code

In [190]:
archive_clean.retweeted_status_timestamp = pd.to_datetime(archive_clean.retweeted_status_timestamp)

#### Test

In [191]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2058 entries, 0 to 2058
Data columns (total 16 columns):
tweet_id                      2058 non-null int64
in_reply_to_status_id         2058 non-null object
in_reply_to_user_id           2058 non-null object
timestamp                     2058 non-null datetime64[ns, UTC]
source                        2058 non-null object
text                          2058 non-null object
retweeted_status_id           2058 non-null object
retweeted_status_user_id      2058 non-null object
retweeted_status_timestamp    72 non-null datetime64[ns, UTC]
expanded_urls                 2058 non-null object
rating_numerator              2058 non-null int64
rating_denominator            2058 non-null int64
name                          2058 non-null object
retweet_count                 2058 non-null int64
favorite_count                2058 non-null int64
stage                         2058 non-null object
dtypes: datetime64[ns, UTC](2), int64(5), object(9)
memory

<a id='eda'></a>
## Exploratory Data Analysis 

## Conclusions

## Limitations